In [1]:
# import utensor_cgen as test
from utensor_cgen.api.export import tflm_keras_export
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
print(tf.__version__)

2.1.0


In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

In [4]:
# Shuffling the dataset and forming batches
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [5]:
model = keras.Sequential()
model.add(tf.keras.Input(shape=(28, 28, 1)))  # 250x250 RGB images

model.add(layers.Conv2D(32, kernel_size=(3, 3), activation="relu") )
model.add(layers.MaxPooling2D(pool_size=(2, 2)) )
model.add(layers.Conv2D(32, kernel_size=(3, 3), activation="relu"))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(10, activation='linear'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 800)               0         
_________________________________________________________________
dense (Dense)                (None, 10)                8010      
Total params: 17,578
Trainable params: 17,578
Non-trainable params: 0
____________________________________________________

## Defining the Model

## Loss Functions, and Optimisers

In [6]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

In [7]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

## Defining Training and Testing Steps

#### Training Step

In [8]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

#### Model Test Step

In [9]:
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

## Training Loop

In [10]:
EPOCHS = 10

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print(template.format(epoch+1,
                        train_loss.result(),
                        train_accuracy.result()*100,
                        test_loss.result(),
                        test_accuracy.result()*100))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1, Loss: 0.17661736905574799, Accuracy: 94.7266616821289, Test Loss: 0.05875467509031296, Test Accuracy: 98.15999603271484
Epoch 2, Loss: 0.06266306340694427, Accuracy: 98.08999633789062, Test Loss: 0.04921255633234978, Test Accuracy: 98.3499984741211
Epoch 3, Loss: 0.04800419881939888, Accuracy: 98.48999786376953, Test Loss: 0.03858775645494461, Test Accuracy: 98.70999908447266
Epoch 4, Loss: 0.038669705390930176, Accuracy: 98.788330078125, Test Loss: 0.03530162200331688, Test Accuracy: 98.809997

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 800)               0         
_________________________________________________________________
dense (Dense)                (None, 10)                8010      
Total params: 17,578
Trainable params: 17,578
Non-trainable params: 0
____________________________________________________

## Exporting to uTensor

In [12]:

# representative data function
num_calibration_steps = 128
calibration_dtype = tf.float32

def representative_dataset_gen():
    for _ in range(num_calibration_steps):
        rand_idx = np.random.randint(0, x_test.shape[0]-1)
        sample = x_test[rand_idx]
        sample = sample[tf.newaxis, ...]
        sample = tf.cast(sample, dtype=calibration_dtype)
        yield [sample]

In [13]:
!rm -R ./constants/myconv
!rm -R ./models/myconv

One-liner uTensor export API

In [14]:
tflm_keras_export(
    model,
    representive_dataset=representative_dataset_gen,
    model_name='myconv',
    target='utensor',
)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: /var/folders/mv/1txqzr191f1b8gyq0sjcwyth0000gn/T/utensor_4il4atkl/saved_model/assets
[WARNING base.py __attrs_post_init__ @ 300] cannot convert new_shape to generic value: [1, 800](<class 'list'>)
[WARNING quantize.py <module> @ 12] trying to import deprecated quantization transformer
[INFO transformer.py transform @ 23] Transforming graph: myconv
[INFO transformer.py transform @ 24] Transform pipeline: dropout(name_pattern=r'(dropout[_\w\d]*)/.*') -> linear_reorder -> inline -> biasAdd -> remove_id_op -> fake_gather_v2 -> refcnt
[WARNING base.py __attrs_post_init__ @ 300] cannot convert new_shape to generic value: [1, 800](<class 'list'>)
[WARNING base.py __attrs_post_init__ @ 300] cannot convert new_shape to generic value: [1, 800](<class 'list'>)
[WARNING base.py __attrs_post_init__ @ 300] cannot convert new_shape to generic value: [1, 800](<class 'list'>)
[WARNING bas

In [15]:
!ls ./models/

myconv
